In [1]:
import numpy as np
from cppy import *
from cppy.model_tools.to_cnf import *

In [2]:
# sudoku's, from http://hakank.org/minizinc/sudoku_problems2/index.html

sudoku_p0 = np.array([[0,0,0, 2,0,5, 0,0,0],
                      [0,9,0, 0,0,0, 7,3,0],
                      [0,0,2, 0,0,9, 0,6,0],
                      [2,0,0, 0,0,0, 4,0,9],
                      [0,0,0, 0,7,0, 0,0,0],
                      [6,0,9, 0,0,0, 0,0,1],
                      [0,8,0, 4,0,0, 1,0,0],
                      [0,6,3, 0,0,0, 0,8,0],
                      [0,0,0, 6,0,8, 0,0,0]])

# trivial version
sudoku_p0b = np.array([[0, 7, 8, 2, 6, 5, 9, 1, 4],
       [5, 9, 6, 8, 1, 4, 7, 3, 2],
       [1, 4, 2, 7, 3, 9, 5, 6, 8],
       [2, 1, 7, 3, 8, 6, 4, 5, 9],
       [8, 5, 4, 9, 7, 1, 6, 2, 3],
       [6, 3, 9, 5, 4, 2, 8, 7, 1],
       [7, 8, 5, 4, 2, 3, 1, 9, 6],
       [4, 6, 3, 1, 9, 7, 2, 8, 5],
       [9, 2, 1, 6, 5, 8, 3, 4, 7]])

In [3]:
from ortools.sat.python import cp_model

# model and solve a sudoku with ortools
def model_sudoku_ort(grid):
        csp = cp_model.CpModel()

        # init vars
        board = [[csp.NewIntVar(1, 9, 'x_%i%i' % (i,j)) for j in range(9)] for i in range(9)]
        
        # assign knowns
        for i in range(9):
            for j in range(9):
                if grid[i][j] != 0:
                    csp.Add(board[i][j] == grid[i][j])
        
        # all different rows
        for i in range(9):
            csp.AddAllDifferent(board[i])
        
        # all different columns
        for j in range(9):
            csp.AddAllDifferent([board[i][j] for i in range(9)])
        
        # all different cells
        for si in range(3):
            for sj in range(3):
                csp.AddAllDifferent([board[3*si+i][3*sj+j] for j in range(3) for i in range(3)])
        
        return csp, board
def solve_sudoku_ort(grid):
    # the constraint model and decision vars
    csp, x = model_sudoku_ort(grid)
    
    solver = cp_model.CpSolver()
    status = solver.Solve(csp) # or similar?
    solution = np.zeros((9,9), dtype=int)
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        for i in range(9):
            for j in range(9):
                solution[i][j] = solver.Value(x[i][j])
        return solution 
    else:
        print("CSP infeasible")

sol = solve_sudoku_ort(sudoku_p0.tolist()) # grid must be plain python lists
sol

array([[3, 7, 8, 2, 6, 5, 9, 1, 4],
       [5, 9, 6, 8, 1, 4, 7, 3, 2],
       [1, 4, 2, 7, 3, 9, 5, 6, 8],
       [2, 1, 7, 3, 8, 6, 4, 5, 9],
       [8, 5, 4, 9, 7, 1, 6, 2, 3],
       [6, 3, 9, 5, 4, 2, 8, 7, 1],
       [7, 8, 5, 4, 2, 3, 1, 9, 6],
       [4, 6, 3, 1, 9, 7, 2, 8, 5],
       [9, 2, 1, 6, 5, 8, 3, 4, 7]])

In [4]:
# model and sudoku with cppy
given = sudoku_p0

# Variables
n = given.shape[0]
puzzle = IntVar(1, n, shape=given.shape)

constraint = []
# constraints on rows and columns
constraint += [ alldifferent(row) for row in puzzle ]
constraint += [ alldifferent(col) for col in puzzle.T ]

# constraint on blocks
for i in range(0,n,3):
    for j in range(0,n,3):
        constraint += [ alldifferent(puzzle[i:i+3, j:j+3]) ]

# constraints on values
constraint += [ puzzle[given>0] == given[given>0] ]

model = Model(constraint)
#stats = model.solve() # can't run minizinc in notebook
#print(stats)
#print(puzzle.value())


# We are solving the trivial case here!

In [5]:
# model and sudoku with cppy, Boolean style
given = sudoku_p0b

hardcons = []
softlits = []
givens = []

# Variables
n = given.shape[0]
puzzle = BoolVar(shape=(n,n,n)) # last dimension is values 1..9

# boolvars of givens
givens = []
for r in range(n):
    for c in range(n):
        if given[r,c] > 0:
            givens += [puzzle[r,c,given[r,c]-1]] # the positive
            givens += [~puzzle[r,c,e] for e in range(n) if e != given[r,c]-1] # the negatives       
#givens = [puzzle[r,c,given[r,c]-1] for r in range(n) for c in range(n) if given[r,c] > 0] # -1 cus offset 0
print(givens)

# each cell a unique value
bij_exists = [any(puzzle[i,j,:]) for i in range(n) for j in range(n)]
bij_nondbl = [(~puzzle[r,c,e1] | ~puzzle[r,c,e2]) for e1 in range(n) for e2 in range(e1+1,n) for r in range(n) for c in range(n)]
i1 = BoolVar()
hardcons += to_cnf([implies(i1, bijX) for bijX in bij_exists+bij_nondbl])
softlits += [i1]
print("grps:", len(softlits))

# each row different
for r in range(n):
    ralldiff = [any(puzzle[r,:,e]) for e in range(n)]
    ralldiff += [(~puzzle[r,j1,e] | ~puzzle[r,j2,e]) for e in range(n) for j1 in range(n) for j2 in range(j1,n) if j1 != j2]
    ri = BoolVar()
    hardcons += to_cnf([implies(ri, cons) for cons in ralldiff])
    softlits += [ri]
print("grps:", len(softlits))

# each col different
for c in range(n):
    calldiff = [any(puzzle[:,c,e]) for e in range(n)]
    calldiff += [(~puzzle[i1,c,e] | ~puzzle[i2,c,e]) for e in range(n) for i1 in range(n) for i2 in range(i1,n) if i1 != i2]
    ci = BoolVar()
    hardcons += to_cnf([implies(ci, cons) for cons in calldiff])
    softlits += [ci]
print("grps:", len(softlits))

## each block different
for i in range(0,n,3):
    for j in range(0,n,3):
        cblock = [any(puzzle[i1,j1,e] for i1 in range(i,i+3) for j1 in range(j,j+3)) for e in range(n)]
        cblock += [(~puzzle[i1,j1,e] | ~puzzle[i2,j2,e]) for e in range(n) for i1 in range(i,i+3) for i2 in range(i,i+3) for j1 in range(j,j+3) for j2 in range(j,j+3) if not(i1 == i2 and j1 == j2)]
        bi = BoolVar()
        hardcons += to_cnf([implies(bi, cons) for cons in cblock])
        softlits += [bi]
print("grps:", len(softlits))

[BV15, BV9 == 0, BV10 == 0, BV11 == 0, BV12 == 0, BV13 == 0, BV14 == 0, BV16 == 0, BV17 == 0, BV25, BV18 == 0, BV19 == 0, BV20 == 0, BV21 == 0, BV22 == 0, BV23 == 0, BV24 == 0, BV26 == 0, BV28, BV27 == 0, BV29 == 0, BV30 == 0, BV31 == 0, BV32 == 0, BV33 == 0, BV34 == 0, BV35 == 0, BV41, BV36 == 0, BV37 == 0, BV38 == 0, BV39 == 0, BV40 == 0, BV42 == 0, BV43 == 0, BV44 == 0, BV49, BV45 == 0, BV46 == 0, BV47 == 0, BV48 == 0, BV50 == 0, BV51 == 0, BV52 == 0, BV53 == 0, BV62, BV54 == 0, BV55 == 0, BV56 == 0, BV57 == 0, BV58 == 0, BV59 == 0, BV60 == 0, BV61 == 0, BV63, BV64 == 0, BV65 == 0, BV66 == 0, BV67 == 0, BV68 == 0, BV69 == 0, BV70 == 0, BV71 == 0, BV75, BV72 == 0, BV73 == 0, BV74 == 0, BV76 == 0, BV77 == 0, BV78 == 0, BV79 == 0, BV80 == 0, BV85, BV81 == 0, BV82 == 0, BV83 == 0, BV84 == 0, BV86 == 0, BV87 == 0, BV88 == 0, BV89 == 0, BV98, BV90 == 0, BV91 == 0, BV92 == 0, BV93 == 0, BV94 == 0, BV95 == 0, BV96 == 0, BV97 == 0, BV104, BV99 == 0, BV100 == 0, BV101 == 0, BV102 == 0, BV103 

In [7]:
def visu_int2d(m, puzzle):
    boolarr = np.array([b in m for b in [e for lst in cnf_to_pysat(puzzle.tolist()) for e in lst]]).reshape(puzzle.shape)
    n = puzzle.shape[0]
    mtrx = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            idx = np.where(boolarr[i,j])[0]
            #print(idx)
            if len(idx) != 1:
                print(f"Err: m[{i},{j}] has {len(idx)} true values")
            else:
                mtrx[i,j] = idx[0]+1
    print(mtrx)

In [8]:
# lets solve it...
# pysat imports
from pysat.formula import CNF
from pysat.solvers import Solver

cnf = cnf_to_pysat(softlits + givens + hardcons)
with Solver(bootstrap_with=cnf) as s:
    sat = s.solve()
    print(sat)
    prev = None
    for id, m in enumerate(s.enum_models()):
            print(f"{id}: model found")
            visu_int2d(m, puzzle)
            #print(np.array(m).reshape(n,n,n))
            if not prev is None:
                print("diff", sorted(set(prev) - set(m), key=lambda l: abs(l)))
                break
            prev = m


True
0: model found
[[3. 7. 8. 2. 6. 5. 9. 1. 4.]
 [5. 9. 6. 8. 1. 4. 7. 3. 2.]
 [1. 4. 2. 7. 3. 9. 5. 6. 8.]
 [2. 1. 7. 3. 8. 6. 4. 5. 9.]
 [8. 5. 4. 9. 7. 1. 6. 2. 3.]
 [6. 3. 9. 5. 4. 2. 8. 7. 1.]
 [7. 8. 5. 4. 2. 3. 1. 9. 6.]
 [4. 6. 3. 1. 9. 7. 2. 8. 5.]
 [9. 2. 1. 6. 5. 8. 3. 4. 7.]]


In [9]:
# lets explain it...
import explain

def flat(ll):
    return [e for l in ll for e in l]

params = explain.COusParams()
params.pre_seeding = True
params.polarity = True
params.grow = True
params.grow_maxsat = True
params.grow_maxsat_unit = True

cnfcnf = CNF(from_clauses=cnf_to_pysat(hardcons))
I = set(flat(cnf_to_pysat(givens+softlits)))
user_pos = set(flat(cnf_to_pysat(puzzle.flat)))
user_vars = I | user_pos | set([-e for e in user_pos])
# needs much better cost function: positive puzzles cheaper than negative, constraints more exp, bij cheaper than cons
f = lambda x: 1 # all unit

explain.explain(C=cnfcnf, U=user_vars, f=f, I0=I, params=params, verbose=True)

Expl:
	cnf: 14904 757
	U: 1486
	f: <function <lambda> at 0x7f2e9648af28>
	I0: 748
{3, 16, 26, 29, 42, 50, 63, 64, 76, 86, 99, 105, 116, 118, 130, 142, 147, 155, 163, 175, 182, 196, 201, 216, 221, 231, 242, 245, 253, 268, 273, 287, 294, 301, 311, 324, 332, 338, 346, 360, 367, 370, 384, 389, 399, 411, 417, 432, 437, 445, 452, 467, 475, 478, 493, 503, 509, 517, 524, 534, 541, 558, 564, 571, 582, 588, 595, 612, 619, 623, 638, 644, 657, 659, 667, 681, 689, 701, 705, 715, 727, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, -729, -728, -726, -725, -724, -723, -722, -721, -720, -719, -718, -717, -716, -714, -713, -712, -711, -710, -709, -708, -707, -706, -704, -703, -702, -700, -699, -698, -697, -696, -695, -694, -693, -692, -691, -690, -688, -687, -686, -685, -684, -683, -682, -680, -679, -678, -677, -676, -675, -674, -673, -672, -671, -670, -669, -668, -666, -665, -664, -663, -662, -661, -660, -658, 


Optimal explanation 		 {105, 749} => {-6} 	 (cost: 2)

	MODE_OPT: got HS 5 cost 5.0 	MIP: 0.005 s	GROW: 0
	MODE_OPT: got HS 6 cost 6.0 	MIP: 0.008 s	GROW: 0.073
	MODE_OPT: got HS 6 cost 6.0 	MIP: 0.005 s	GROW: 0.078
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.014 s	GROW: 0.053
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.009 s	GROW: 0.055
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.008 s	GROW: 0.055
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.009 s	GROW: 0.044
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.007 s	GROW: 0.05
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.011 s	GROW: 0.058
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.009 s	GROW: 0.044
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.01 s	GROW: 0.051
	MODE_OPT: got HS 7 cost 7.0 	MIP: 0.007 s	GROW: 0.089
	MODE_OPT: got HS 8 cost 8.0 	MIP: 0.053 s	GROW: 0.118
	MODE_OPT: got HS 8 cost 8.0 	MIP: 0.026 s	GROW: 0.116
	MODE_OPT: got HS 8 cost 8.0 	MIP: 0.044 s	GROW: 0.051
	MODE_OPT: got HS 8 cost 8.0 	MIP: 0.025 s	GROW: 0.053
	MODE_OPT: got HS 8 cost 8.0 	MIP: 0.026 s	GROW: 0.071
	MODE_OPT: got 

[{'constraints': [64, 731], 'derived': [-1]},
 {'constraints': [749, 86], 'derived': [-5]},
 {'constraints': [749, 175], 'derived': [-4]},
 {'constraints': [731, 29], 'derived': [-2]},
 {'constraints': [16, 749], 'derived': [-7]},
 {'constraints': [657, 740], 'derived': [-9]},
 {'constraints': [26, 749], 'derived': [-8]},
 {'constraints': [105, 749], 'derived': [-6]},
 {'constraints': [201, -21, 749, 147, -12, 732, 733], 'derived': [3]}]

In [10]:
# the helper function below DOES NOT WORK, to be done in the CPpy future perhaps...
def alldifferent_bool(m):
    # assumption: dim0 = different variables, dim1 = boolvar for each value
    (nv,nb) = m.shape
    return [(~puzzle[j1,e] | ~puzzle[j2,e]) for e in range(nb) for j1 in range(nv) for j2 in range(nv) if j1 != j2]
#alldifferent_bool(puzzle[r,:,:])